# Optofluidics Template
This Jupyter Notebook is a template for using the Optofluidics module to process HDF5 spectroscopic data.

In [ ]:
import optofluidics as of
from lmfit import Parameters
import matplotlib.pyplot as plt

# Set file path to datafile location, including the .h5 file extension.
file_path = r'C:\Users\tl457\OneDrive - University Of Cambridge 1\Measurements\Carbon Dots - Absorption\Jupyter Temp\2019-11-06.h5'

# Set save path to the desired export location.
export_path = r'C:\Users\tl457\OneDrive - University Of Cambridge 1\Measurements\Carbon Dots - Absorption\Exports'

## Load Datafile
Load the desired datafile and list the datasets contained within.

In [ ]:
# Load the datafile and create a Datafile instance
df1=of.Datafile(file_path)

# List the datasets contained within the datafile
df1.list_groups()

## Pre-Processing
Create a Dataset instance, pre-process the data to calculate elapsed time and background correct, then calculate the absorbances using the stored reference spectra.

In [ ]:
# Create a Dataset instance (default is key 0 and timelapse 0 for the datafile specified)
ds1=of.Dataset(df1,0,0)

# Pre-process the data and calculate the absorbances
ds1.pre_process()
ds1.calculate_abs()

# Export absorbance data to .csv
# ds1.abs2csv('2019-11-06-abs.csv',export_path)

In [ ]:
# Plot all absorbance spectra
of.plot_abs(ds1)

# Plot absorbance chnages over time at specific wavelengths
of.wav_abs(ds1,[600],plot=True)

# Plot absorbance spectra at specific times
of.time_abs(ds1,[1000],plot=True)

## Processing
Plot colourmap, calculate concentrations and apply a drift correction

In [ ]:
# Plot colourmap
of.colourplot(ds1,[1000],[600],-0.5,1.5)

In [ ]:
# Create Reaction instance
c1=of.Reaction(ds1,600,13600,5)

# Calculate concentrations
c1.calculate_conc(5) # 5 is the wavelength range to average over (i.e. 600-2.5 to 600+2.5)

# Apply a linear drift correction
c1.linear_drift([2790,5750,8650])

# Plot the concentration profile
c1.plot_conc()

In [ ]:
# Find turning points to automatically detect the UV on/off times
# Ignore the first turning point in the plot as we manually specify in the find_turn_points function

times_arr=c1.find_turn_points(182,0.022,translate=60,plot=True)
print(c1.turning_points)

## Modelling
Create an initial model then run a fitting algorithm to find the best parameters

In [ ]:
# define initial model parameters ahead of fitting
params = Parameters()
params.add('t_erf', value = 231, min=0, max=500) #413
params.add('kc', value = 0.01, min=1e-5, max=0.1, vary = False) # raises NaN if value > 0.1
params.add('kbr', value = 0.000457, min=0.0001, max=0.1) #0.000457
params.add('k', value = 0.00980, min=0.0001, max=0.1) #0.00980
params.add('kr', value = 0.00369, min=0.0001, max=0.1) #0.00369
params.add('o', value = 68, min=10, max=100) #68
params.add('c0', value = 40, vary = False)
params.add('K', value = 0.786, vary = False) # set K=0 for no complexation; set K=0.786 for complexation
params.add('end', value = 9000, vary = False)

# create the model and plot
model=c1.create_model(params)
of.plot_model(c1)

In [ ]:
# Run fitting algorithm (see lmfit documentation)
c1.fit_model(params,meth='leastsq',save_path=export_path) # fitting report saved to save_path

# Plot a comparison of the experimental data to the model
of.compare_model(c1)

In [ ]:
# Plot rates
of.plot_rates(c1)

# Focus on carbon dot activation region
rates = c1.get_rates()
plt.plot(rates['k(t)'])
plt.title('k(t)')
plt.xlim(100,1000);
plt.ylim(0,0.01);

In [ ]:
# Export fitted model to csv file
c1.model2csv('2019-11-06-model.csv',export_path)